# ENTENDIENDO EL MÓDULO "LSTM" EN KERAS

En este tutorial vamos a entender cómo usar las principales opciones del módulo "LSTM" de Keras para la creación de Redes LSTM.

Contenido:
1. [Redes LSTM: breve repaso](#scrollTo=RemDAOPx5auE&line=1&uniqifier=1)
2. [Keras y la celda LSTM básica](#scrollTo=CnqQ4VlH_WoL&line=10&uniqifier=1)
3. [La opción "return_sequences"](#scrollTo=3hMNGse7BFKn&line=19&uniqifier=1)
4. [La opción "return_state"](#scrollTo=J6fhsNIiE4jF&line=14&uniqifier=1)
5. [Usando "return_sequences" y "return_states" simultáneamente](#scrollTo=3DSMNdaOJKXr&line=1&uniqifier=1)
6. [Celda LSTM + capa "Dense"](#scrollTo=-ktFF1d-4ldC&line=9&uniqifier=1)

## 1. Redes LSTM: breve repaso

La Red LSTM tiene dos elementos centrales:

- El estado oculto, que es como la **memoria de corto plazo** y
- La celda de estado, que es como la **memoria de largo plazo**

![](https://drive.google.com/uc?export=view&id=1D-KyGNut7oVSMgQChiIRiJ1uLSih4omw)

Así que durante el entrenamiento la Red LSTM aprende a determinar qué información de corto y largo plazo es relevante para interpretar correctamente la secuencia.

## 2. Keras y la celda LSTM básica

En Keras una Red LSTM básica se conoce como una celda.

En su versión por defecto la creamos definiendo dos parámetros:

- `input_shape`: el tamaño de cada dato de entrada (*timesteps* x *features*)
- `units`: el número de unidades de la celda. Este número de unidades definirá a su vez la complejidad (número de parámetros) de la celda así como el tamaño del dato de salida (*units*)

![](https://drive.google.com/uc?export=view&id=1Ej1XGizCIGIkzvebVW1S1thNVeA5H03B)


Veamos un ejemplo sencillo. Creemos una celda LSTM con estas características:

- Entradas: secuencias de 3 elementos y 1 feature (`input_shape` de 3x1)
- Unidades: 5 (`units`)

Veamos cómo implementar esta celda:

In [ ]:
# Importar modulos requeridos
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM
from tensorflow.random import set_seed
import numpy as np
import random

# Función fijar semillas (para reproducibilidad de los resultados)
def fijar_semillas():
    set_seed(123)
    np.random.seed(123)
    random.seed(123)

# Crear celda LSTM:
# - input_shape de 3x1
# - units = 5
fijar_semillas()
entrada = Input(shape=(3,1)) # (timesteps = 3) x (features = 1)
lstm_out = LSTM(5)(entrada) # Celda LSTM con 5 unidades
modelo = Model(inputs=entrada, outputs=lstm_out)

# Imprimir información del modelo
print('Información general del modelo: ')
print('-'*70)
print(modelo.summary())

print('-'*70)
print('Tamaño entrada: ', modelo.input_shape)
print('Tamaño de salida:', modelo.output_shape)

# Generar predicción e imprimir resultado en pantalla
datos = np.array([0.5, 0.4, 0.3]).reshape((1,3,1)) # 1 dato con 3 timesteps y 1 feature
pred = modelo.predict(datos, verbose=0)

print('-'*70)
print('Predicción (h_t): ', pred)
print('Tamaño predicción: ', pred.shape)

Información general del modelo: 
----------------------------------------------------------------------
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm_2 (LSTM)               (None, 5)                 140       
                                                                 
Total params: 140 (560.00 Byte)
Trainable params: 140 (560.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
----------------------------------------------------------------------
Tamaño entrada:  (None, 3, 1)
Tamaño de salida: (None, 5)
----------------------------------------------------------------------
Predicción (h_t):  [[ 0.04174358 -0.06729254  0.03022056 -0.05514956  0.02848158]]
Tamaño predicción:  (1, 5

Vemos que la predicción (salida del modelo) tiene exactamente 5 elementos, el mismo número de unidades de la celda LSTM.

Si ahora la entrada contiene 2 y no 1 *feature*, la salida seguirá teniendo 5 elementos:

In [ ]:
# Crear celda LSTM
# - input_shape = 3 (timesteps) x 2 (features)
# - units = 5
fijar_semillas()
entrada = Input(shape=(3,2))
lstm_out = LSTM(5)(entrada)
modelo = Model(inputs=entrada, outputs=lstm_out)

# Imprimir información del modelo
print('Información general del modelo: ')
print('-'*70)
print(modelo.summary())

print('-'*70)
print('Tamaño entrada: ', modelo.input_shape)
print('Tamaño de salida:', modelo.output_shape)

# Generar predicción e imprimir resultado en pantalla
datos = np.array([[0.5, 0.4, 0.3],
                  [0.6, 0.9, 0.8]]).reshape((1,3,2))
pred = modelo.predict(datos, verbose=0)

print('-'*70)
print('Predicción (h_t): ', pred)
print('Tamaño predicción: ', pred.shape)

Información general del modelo: 
----------------------------------------------------------------------
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 3, 2)]            0         
                                                                 
 lstm_3 (LSTM)               (None, 5)                 160       
                                                                 
Total params: 160 (640.00 Byte)
Trainable params: 160 (640.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
----------------------------------------------------------------------
Tamaño entrada:  (None, 3, 2)
Tamaño de salida: (None, 5)
----------------------------------------------------------------------
Predicción (h_t):  [[-0.09150496  0.05164478  0.19230194  0.07269354 -0.10549309]]
Tamaño predicción:  (1, 5

## 3. La opción `return_sequences`

En los ejemplos anteriores hemos usado las opciones por defecto.

En particular, una de esas opciones por defecto es `return_sequences = False`.

Con esta opción la salida de la celda únicamente contendrá la predicción correspondiente **último dato en la secuencia de entrada**:

| *timestep* (entrada) | valor | salida ($h_t$)                   |
|:--------------------:|:-----:|----------------------------------|
| 1                    | 0.5   | -------------                              |
| 2                    | 0.4   | -------------                              |
| 3                    | 0.3   | [0.04, -0.06, 0.03, -0.05, 0.02] |


Sin embargo, si hacemos `return_sequences=True` la celda retornará una predicción **por cada elemento en la secuencia de entrada**:

![](https://drive.google.com/uc?export=view&id=1H2sadt1RT46C2hJvdxvh6YloqCwMklsd)


Es decir que ahora la salida será de tamaño *timesteps* (el tamaño de la secuencia de entrada) x *units* (el número de unidades de la celda LSTM).

Veamos el mismo caso del primer ejemplo (entrada de 3x1) pero ahora con `return_sequences=True`):

In [ ]:
# Crear celda LSTM
# - input_shape = 3x1
# - units = 5
# - return_sequences = True
fijar_semillas()
entrada = Input(shape=(3,1))
lstm_out = LSTM(5,
                return_sequences=True)(entrada) # *** Celda LSTM con return_sequences=True
modelo = Model(inputs=entrada, outputs=lstm_out)

# Imprimir información del modelo
print('Información general del modelo: ')
print('-'*70)
print(modelo.summary())

print('-'*70)
print('Tamaño entrada: ', modelo.input_shape)
print('Tamaño de salida:', modelo.output_shape)

# Generar predicción e imprimir resultado en pantalla
datos = np.array([0.5, 0.4, 0.3]).reshape((1,3,1))
pred = modelo.predict(datos, verbose=0)

print('-'*70)
print('Predicción (h_t): ', pred)
print('Tamaño predicción: ', pred.shape)

Información general del modelo: 
----------------------------------------------------------------------
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm_4 (LSTM)               (None, 3, 5)              140       
                                                                 
Total params: 140 (560.00 Byte)
Trainable params: 140 (560.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
----------------------------------------------------------------------
Tamaño entrada:  (None, 3, 1)
Tamaño de salida: (None, 3, 5)


----------------------------------------------------------------------
Predicción (h_t):  [[[ 0.02129798 -0.04037673  0.02123253 -0.02901676  0.01275784]
  [ 0.03483645 -0.0609595   0.02958979 -0.04655094  0.02243249]
  [ 0.04174358 -0.06729254  0.03022056 -0.05514956  0.02848158]]]
Tamaño predicción:  (1, 3, 5)


Así que veamos la diferencia entre `return_sequences=False` y `return_sequences=True`:

| *timestep* (entrada) | valor | salida ($h_t$) sin "return_sequences" | salida ($h_t$) con "return_sequences" |
|:--------------------:|:-----:|---------------------------------------|---------------------------------------|
| 1                    | 0.5   | -------------                                   | [0.02, -0.04, 0.02, -0.02, 0.01]      |
| 2                    | 0.4   | -------------                                   | [0.03, -0.06, 0.02, -0.04, 0.02]      |
| 3                    | 0.3   | [0.04, -0.06, 0.03, -0.05, 0.02]      | [0.04, -0.06, 0.03, -0.05, 0.02]      |

Es decir que:

> `return_sequences=True` retorna **todos** los estados ocultos

Esta opción de `return_sequences=True` la podemos usar por ejemplo cuando queremos usar múltiples celdas LSTM (es decir conectar una después de la otra).

## 4. La opción `return_state`

Hasta este punto hemos visto que la Red LSTM únicamente retorna el estado oculto ($h_t$) pero no la celda de estado ($c_t$).

Si queremos retornar el último estado oculto y el último valor de la celda de estado, debemos usar `return_states=True`:

![](https://drive.google.com/uc?export=view&id=1GpgZjw78ykBE2UT3FEUYiYaKTpOH0HAy)

En este caso la celda retorna 2 salidas:

- El último estado oculto $h_t$
- El último valor en la celda de estado $c_t$

Por ejemplo, creemos la misma celda LSTM del primer ejemplo (entrada 3x1, 5 unidades) pero ahora usemos `return_state=True`:

In [ ]:
# Crear celda LSTM
# - input_shape = 3x1
# - units = 5
# - return_state = True
fijar_semillas()
entrada = Input(shape=(3,1)) # (timesteps = 3) x (features = 1)
lstm_out, h_t, c_t = LSTM(5,
                          return_state=True)(entrada) # ***Celda LSTM con return_state=True***
modelo = Model(inputs=entrada, outputs=[lstm_out, h_t, c_t])

# Imprimir información del modelo
print('Información general del modelo: ')
print('-'*70)
print(modelo.summary())

print('-'*70)
print('Tamaño entrada: ', modelo.input_shape)
print('Tamaño de salida:', modelo.output_shape)

# Generar predicción e imprimir resultado en pantalla
datos = np.array([0.5, 0.4, 0.3]).reshape((1,3,1))
pred, h, c = modelo.predict(datos, verbose=0)

print('-'*70)
print('Predicción (lstm_out): ', pred)
print('Tamaño predicción "lstm_out": ', pred.shape)
print('.'*50)
print('Predicción (h_t): ', h)
print('Tamaño predicción "h_t": ', h.shape)
print('.'*50)
print('Predicción (c_t): ', c)
print('Tamaño predicción "c_t": ', c.shape)

Información general del modelo: 
----------------------------------------------------------------------
Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm_5 (LSTM)               [(None, 5),               140       
                              (None, 5),                         
                              (None, 5)]                         
                                                                 
Total params: 140 (560.00 Byte)
Trainable params: 140 (560.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
----------------------------------------------------------------------
Tamaño entrada:  (None, 3, 1)
Tamaño de salida: [(None, 5), (None, 5), (None, 5)]


----------------------------------------------------------------------
Predicción (lstm_out):  [[ 0.04174358 -0.06729254  0.03022056 -0.05514956  0.02848158]]
Tamaño predicción "lstm_out":  (1, 5)
..................................................
Predicción (h_t):  [[ 0.04174358 -0.06729254  0.03022056 -0.05514956  0.02848158]]
Tamaño predicción "h_t":  (1, 5)
..................................................
Predicción (c_t):  [[ 0.08437636 -0.12790795  0.05566084 -0.11978056  0.05761085]]
Tamaño predicción "c_t":  (1, 5)


Es decir que en este caso tendremos:

| *timestep* (entrada) | valor | $h_t$ | $c_t$ |
|:--------------------:|:-----:|---------------------------------------|---------------------------------------|
| 1                    | 0.5   | -------------                                   | -------------      |
| 2                    | 0.4   | -------------                                   | -------------      |
| 3                    | 0.3   | [0.04, -0.06, 0.03, -0.05, 0.02]      | [0.08, -0.12, 0.05, -0.11, 0.05]      |

Esta opción es útil cuando procesamos secuencias relativamente largas y queremos preservar la memoria de largo plazo.

## 5. Usando `return_sequences` y `return_state` simultáneamente

Y finalmente podemos combinar `return_sequences=True` y `return_state=True` para retornar:

- **Todos** los valores de la salida
- El **último** estado oculto ($h_t$)
- El **último** valor de la celda de estados ($c_t$):

Veamos esta implementación:

In [ ]:
# Crear celda LSTM
# - input_shape = 3x1
# - units = 5
# - return_sequences = True
# - return_states = True
fijar_semillas()
entrada = Input(shape=(3,1))
lstm_out, h, c = LSTM(5,
                      return_sequences=True,
                      return_state=True)(entrada) # Celda LSTM
modelo = Model(inputs=entrada, outputs=[lstm_out, h, c])

# Imprimir información del modelo
print('Información general del modelo: ')
print('-'*70)
print(modelo.summary())

print('-'*70)
print('Tamaño entrada: ', modelo.input_shape)
print('Tamaño de salida:', modelo.output_shape)

# Generar predicción e imprimir resultado en pantalla
datos = np.array([0.5, 0.4, 0.3]).reshape((1,3,1))
pred, h, c = modelo.predict(datos, verbose=0)

print('-'*70)
print('Predicción (lstm_out): ', pred)
print('Tamaño predicción "lstm_out": ', pred.shape)
print('.'*50)
print('Último estado oculto (h_t): ', h)
print('Tamaño último estado oculto "h_t": ', h.shape)
print('.'*50)
print('Último valor celda de estado (c_t): ', c)
print('Tamaño último valor celda de estado "c_t": ', c.shape)

Información general del modelo: 
----------------------------------------------------------------------
Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm_6 (LSTM)               [(None, 3, 5),            140       
                              (None, 5),                         
                              (None, 5)]                         
                                                                 
Total params: 140 (560.00 Byte)
Trainable params: 140 (560.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
----------------------------------------------------------------------
Tamaño entrada:  (None, 3, 1)
Tamaño de salida: [(None, 3, 5), (None, 5), (None, 5)]
--------------------

## 6. Celda LSTM + capa "Dense"

Cuando queremos por ejemplo generar pronósticos sobre Series de Tiempo o clasificar una secuencia (por ejemplo en análisis de sentimientos) debemos añadir una capa de salida a la celda LSTM:

![](https://drive.google.com/uc?export=view&id=1HCNWkAmn_o9QEv_IIzWpeSRs7oIbAYHm)

Por ejemplo, si queremos generar pronósticos o generar texto a la salida, la capa `Dense` debe hacer una tarea de regresión (es decir debe predecir un número). En este caso la función de activación de `Dense` debe ser lineal.

Por ejemplo, este sería el código para predecir un valor a futuro:

In [ ]:
from tensorflow.keras.layers import Dense

# Crear celda LSTM:
# - input_shape de 3x1
# - units = 5
fijar_semillas()
entrada = Input(shape=(3,1)) # (timesteps = 3) x (features = 1)
lstm_out = LSTM(5)(entrada) # Celda LSTM con 5 unidades

# Agregar capa de salida Dense
salida = Dense(1)(lstm_out) # Capa de salida con 1 neurona y función de activación lineal
modelo = Model(inputs=entrada, outputs=salida)

# Imprimir información del modelo
print('Información general del modelo: ')
print('-'*70)
print(modelo.summary())

print('-'*70)
print('Tamaño entrada: ', modelo.input_shape)
print('Tamaño de salida:', modelo.output_shape)

# Generar predicción e imprimir resultado en pantalla
datos = np.array([0.5, 0.4, 0.3]).reshape((1,3,1)) # 1 dato con 3 timesteps y 1 feature
pred = modelo.predict(datos, verbose=0)

print('-'*70)
print('Predicción (regresión): ', pred)
print('Tamaño predicción: ', pred.shape)

Información general del modelo: 
----------------------------------------------------------------------
Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm_7 (LSTM)               (None, 5)                 140       
                                                                 
 dense (Dense)               (None, 1)                 6         
                                                                 
Total params: 146 (584.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
----------------------------------------------------------------------
Tamaño entrada:  (None, 3, 1)
Tamaño de salida: (None, 1)
-----------------------------------------------

Y si queremos predecir múltiples valores simplemente añadimos el número de neuronas requerido a la capa "Dense".

Por ejemplo, este sería el código para predecir 2 valores a futuro:

In [ ]:
# Crear celda LSTM:
# - input_shape de 3x1
# - units = 5
fijar_semillas()
entrada = Input(shape=(3,1)) # (timesteps = 3) x (features = 1)
lstm_out = LSTM(5)(entrada) # Celda LSTM con 5 unidades


# Agregar capa de salida Dense
salida = Dense(2)(lstm_out) # Capa de salida con 2 neuronas y función de activación lineal
modelo = Model(inputs=entrada, outputs=salida)

# Imprimir información del modelo
print('Información general del modelo: ')
print('-'*70)
print(modelo.summary())

print('-'*70)
print('Tamaño entrada: ', modelo.input_shape)
print('Tamaño de salida:', modelo.output_shape)

# Generar predicción e imprimir resultado en pantalla
datos = np.array([0.5, 0.4, 0.3]).reshape((1,3,1)) # 1 dato con 3 timesteps y 1 feature
pred = modelo.predict(datos, verbose=0)

print('-'*70)
print('Predicción (regresión): ', pred)
print('Tamaño predicción: ', pred.shape)

Información general del modelo: 
----------------------------------------------------------------------
Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 3, 1)]            0         
                                                                 
 lstm_8 (LSTM)               (None, 5)                 140       
                                                                 
 dense_1 (Dense)             (None, 2)                 12        
                                                                 
Total params: 152 (608.00 Byte)
Trainable params: 152 (608.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
----------------------------------------------------------------------
Tamaño entrada:  (None, 3, 1)
Tamaño de salida: (None, 2)
-----------------------------------------------

Si queremos predecir una categoría podemos usar "Dense" pero con función de activación "sigmoid" (para máximo 2 categorías) o función de activación "softmax" (para 3 o más categorías).

Por ejemplo, un clasificador de secuencias binario sería similar a este:

In [ ]:
# Crear celda LSTM:
# - input_shape de 3x1
# - units = 5
fijar_semillas()
entrada = Input(shape=(3,1)) # (timesteps = 3) x (features = 1)
lstm_out = LSTM(5)(entrada) # Celda LSTM con 5 unidades

# Agregar capa de salida Dense para clasificación
salida = Dense(1, activation='sigmoid')(lstm_out) # Capa de salida con 2 neuronas y función de activación sigmoidal
modelo = Model(inputs=entrada, outputs=salida)

# Imprimir información del modelo
print('Información general del modelo: ')
print('-'*70)
print(modelo.summary())

print('-'*70)
print('Tamaño entrada: ', modelo.input_shape)
print('Tamaño de salida:', modelo.output_shape)

# Generar predicción e imprimir resultado en pantalla
datos = np.array([0.5, 0.4, 0.3]).reshape((1,3,1)) # 1 dato con 3 timesteps y 1 feature
pred = modelo.predict(datos, verbose=0)

print('-'*70)
print('Predicción (clasificación binaria): ', pred)
print('Tamaño predicción: ', pred.shape)

Información general del modelo: 
----------------------------------------------------------------------
Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 3, 1)]            0         
                                                                 
 lstm_9 (LSTM)               (None, 5)                 140       
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 146 (584.00 Byte)
Trainable params: 146 (584.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
----------------------------------------------------------------------
Tamaño entrada:  (None, 3, 1)
Tamaño de salida: (None, 1)
-----------------------------------------------

Y este sería el código de ejemplo para un clasificador de secuencias con 4 categorías:

In [ ]:
# Crear celda LSTM:
# - input_shape de 3x1
# - units = 5
fijar_semillas()
entrada = Input(shape=(3,1)) # (timesteps = 3) x (features = 1)
lstm_out = LSTM(5)(entrada) # Celda LSTM con 5 unidades

# Agregar capa de salida Dense para clasificación
salida = Dense(4, activation='softmax')(lstm_out) # Capa de salida con 2 neuronas y función de activación sigmoidal
modelo = Model(inputs=entrada, outputs=salida)

# Imprimir información del modelo
print('Información general del modelo: ')
print('-'*70)
print(modelo.summary())

print('-'*70)
print('Tamaño entrada: ', modelo.input_shape)
print('Tamaño de salida:', modelo.output_shape)

# Generar predicción e imprimir resultado en pantalla
datos = np.array([0.5, 0.4, 0.3]).reshape((1,3,1)) # 1 dato con 3 timesteps y 1 feature
pred = modelo.predict(datos, verbose=0)

print('-'*70)
print('Predicción (clasificación multiclase): ', pred)
print('Tamaño predicción: ', pred.shape)

Información general del modelo: 
----------------------------------------------------------------------
Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 3, 1)]            0         
                                                                 
 lstm_10 (LSTM)              (None, 5)                 140       
                                                                 
 dense_3 (Dense)             (None, 4)                 24        
                                                                 
Total params: 164 (656.00 Byte)
Trainable params: 164 (656.00 Byte)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
----------------------------------------------------------------------
Tamaño entrada:  (None, 3, 1)
Tamaño de salida: (None, 4)
----------------------------------------------